In [43]:
# Imports

import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [44]:
# Data breakdown: Covid cases versus personal consumption by month
# Import covid case data

covid_path = Path("../Resources/covid_data.csv")
covid_data = pd.read_csv(covid_path)
covid_data.head()



,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,10/15/2020,DC,16166,NaN,NaN,34,0.0,638,NaN,NaN,0,0.0,10/16/2020 01:33:55 PM,NaN,NaN
1,10/28/2020,RI,31944,NaN,NaN,499,0.0,1192,NaN,NaN,4,0.0,10/29/2020 02:11:33 PM,Not agree,NaN
2,08/21/2020,MD,103523,NaN,NaN,624,0.0,3685,3546.0,139.0,11,1.0,08/22/2020 02:21:09 PM,NaN,Agree
3,02/27/2021,MS,294795,182857.0,111938.0,704,478.0,6681,4672.0,2009.0,12,7.0,02/28/2021 02:48:18 PM,Agree,Agree
4,03/12/2020,NJ,29,NaN,NaN,6,NaN,1,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree


In [45]:
# Import personal consumption data
# This was manually cleaned in excel - need to figure out how to clean via python only
# Took the 7th tab "U20305-M" and deleted the top 7 lines

pc_path = Path("../Resources/pc_data.csv")
pc_data = pd.read_csv(pc_path)
pc_data.head(10)

,Line,Unnamed: 1,Unnamed: 2,1959M01,1959M02,1959M03,1959M04,1959M05,1959M06,1959M07,...,2020M05,2020M06,2020M07,2020M08,2020M09,2020M10,2020M11,2020M12,2021M01,2021M02
0,NaN,Personal consumption expenditures by major typ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Personal consumption expenditures (PCE),DPCERC,"306,091","309,554","312,702","312,193","316,130","318,193","317,787",...,"13,165,378","14,014,609","14,224,702","14,397,170","14,582,741","14,627,086","14,532,788","14,451,138","14,939,055","14,790,095"
2,2,Goods,DGDSRC,"167,411","169,596","172,162","170,680","173,326","173,845","172,707",...,"4,447,830","4,745,896","4,815,823","4,859,971","4,929,287","4,922,804","4,862,104","4,772,329","5,174,847","5,018,931"
3,3,Durable goods,DDURRC,"42,295","44,207","44,445","45,065","45,359","45,971","45,408",...,"1,548,706","1,686,563","1,724,514","1,759,846","1,774,640","1,788,684","1,747,593","1,707,931","1,910,545","1,820,253"
4,4,Motor vehicles and parts,DMOTRC,"16,904","18,467","18,900","19,436","19,315","19,254","19,349",...,"527,916","558,120","572,846","581,566","600,472","599,219","578,228","583,676","634,459","605,285"
5,5,Furnishings and durable household equipment,DFDHRC,"15,019","15,286","15,124","15,224","15,465","15,954","15,526",...,"361,862","399,618","400,570","410,714","411,437","413,183","406,650","390,777","445,994","425,673"
6,6,Recreational goods and vehicles,DREQRC,"6,258","6,328","6,284","6,280","6,373","6,470","6,320",...,"491,177","518,616","522,916","530,255","523,954","535,520","528,236","503,355","574,446","542,699"
7,7,Other durable goods,DODGRC,"4,114","4,125","4,137","4,125","4,205","4,293","4,213",...,"167,752","210,208","228,183","237,310","238,777","240,761","234,478","230,122","255,646","246,597"
8,8,Nondurable goods,DNDGRC,"125,117","125,389","127,717","125,615","127,967","127,874","127,299",...,"2,899,124","3,059,334","3,091,309","3,100,126","3,154,646","3,134,121","3,114,511","3,064,399","3,264,303","3,198,677"
9,9,Food and beverages purchased for off-premi...,DFXARC,"61,876","61,379","60,463","60,348","61,562","61,872","61,583",...,"1,148,845","1,147,196","1,152,763","1,141,925","1,146,374","1,138,746","1,151,090","1,126,208","1,192,167","1,174,052"


In [46]:
# Clean covid data by saving only date, state, total cases, and total deaths

covid_df = covid_data.drop(columns=["conf_cases", "prob_cases", "new_case", "pnew_case", "conf_death", "prob_death", "new_death", "pnew_death", "created_at", "consent_cases", "consent_deaths"])

In [47]:
# Sort covid data by date and state

covid_df = covid_df.sort_values(by=['submission_date', 'state'])
covid_df

,submission_date,state,tot_cases,tot_death
13206,01/01/2021,AK,45461,205
4223,01/01/2021,AL,365747,4872
15942,01/01/2021,AR,229442,3711
11162,01/01/2021,AS,3,0
7140,01/01/2021,AZ,530267,9015
6905,01/01/2021,CA,2292568,25971
12111,01/01/2021,CO,337161,4873
22255,01/01/2021,CT,185708,5995
1607,01/01/2021,DC,29252,788
968,01/01/2021,DE,58064,930


In [48]:
# Clean personal consumption data
# Keep columns 2019M12 through 2021M02
# Remove line NaNs, 

pc_df = pc_data.dropna()

In [50]:
pc_df.rename(columns={pc_df.columns[1]: "Category"}, inplace = True)

In [52]:
pc_df = pc_df.drop(columns=pc_df.columns[2]).copy()

In [53]:
pc_df.head()

,Line,Category,1959M01,1959M02,1959M03,1959M04,1959M05,1959M06,1959M07,1959M08,...,2020M05,2020M06,2020M07,2020M08,2020M09,2020M10,2020M11,2020M12,2021M01,2021M02
1,1,Personal consumption expenditures (PCE),"306,091","309,554","312,702","312,193","316,130","318,193","317,787","320,161",...,"13,165,378","14,014,609","14,224,702","14,397,170","14,582,741","14,627,086","14,532,788","14,451,138","14,939,055","14,790,095"
2,2,Goods,"167,411","169,596","172,162","170,680","173,326","173,845","172,707","174,065",...,"4,447,830","4,745,896","4,815,823","4,859,971","4,929,287","4,922,804","4,862,104","4,772,329","5,174,847","5,018,931"
3,3,Durable goods,"42,295","44,207","44,445","45,065","45,359","45,971","45,408","46,189",...,"1,548,706","1,686,563","1,724,514","1,759,846","1,774,640","1,788,684","1,747,593","1,707,931","1,910,545","1,820,253"
4,4,Motor vehicles and parts,"16,904","18,467","18,900","19,436","19,315","19,254","19,349","19,906",...,"527,916","558,120","572,846","581,566","600,472","599,219","578,228","583,676","634,459","605,285"
5,5,Furnishings and durable household equipment,"15,019","15,286","15,124","15,224","15,465","15,954","15,526","15,659",...,"361,862","399,618","400,570","410,714","411,437","413,183","406,650","390,777","445,994","425,673"


In [ ]:
# Setting